In [1]:
from urllib.parse import urlparse
from urllib.parse import urlsplit
import re
import urllib.request
import urllib
import time
from collections import OrderedDict
from bs4 import BeautifulSoup
from bs4.element import Comment
import json
from reppy.robots import Robots



In [91]:
important_words = []
with open("keyword.txt", 'r') as file:
    important_words = file.read().lower().split("\n")[:-1]

In [3]:
# for word in important_words:
#     print(word.lower())

In [4]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

In [5]:
def Canonicalization(parenturl , url):
    
    url = urlsplit(url).geturl()
    url = url.split("?")[0]

    
    if url.find("http") != - 1 or url.find("HTTP") != - 1 or url.find("https") != - 1 or url.find("HTTPs") != -1:
        temp = url.split("//",1)
        temp[1] = temp[1].replace("//" , "/")
        
        if temp[1].find("#") != -1:
            temp[1] = temp[1].split('#')[0]            
        temp_part = temp[1].split("/", 1)

        if temp_part[0].find(":") != -1:
            temp_part[0] = temp_part[0].split(":")[0]
        
        if(len(temp_part) > 1):
            return (temp[0].lower() + "//" + temp_part[0].lower() + "/" + temp_part[1])
        return (temp[0].lower() + "//" + temp_part[0].lower() + "/" )
        
    elif url.find("..") != -1:
        temp = url.split("..")
        temp[1] = temp[1].replace("//" , "/")
        if temp[1].find("#") != -1:
            temp[1] = temp[1].split('#')[0]
        parentTemp = parenturl.split("//")
        parentTemp_part = parentTemp[1].split("/", 1)
        return (parentTemp[0].lower() + "//" + parentTemp_part[0].lower() + temp[1])

    else:
        if url.find("#") != -1:
            url = url.split('#')[0]
            
        if url.find("//") != -1:
            temp = url.split("//",1)    
            pattern = re.compile("^([a-z0-9]+(-[a-z0-9]+)*\.)+[a-z]{2,}$")
            if pattern.match(temp[1].split("/")[0])  != None:
                temp[1] = temp[1].replace("//" , "/")
                if temp[1].find("#") != -1:
                    temp[1] = temp[1].split('#')[0]            
                temp_part = temp[1].split("/", 1)

                if temp_part[0].find(":") != -1:
                    temp_part[0] = temp_part[0].split(":")[0]
                if(len(temp_part) > 1):
                    return ("http:" + "//" + temp_part[0].lower() + "/" + temp_part[1])
                return ("http:" + "//" + temp_part[0].lower())
                

        
        parentTemp = parenturl.split("//")
        parentTemp_part = parentTemp[1].split("/", 1)
        if(url == ""):
            return -1
        return (parentTemp[0].lower() + "//" + parentTemp_part[0].lower() + url)
        



In [7]:
def scrap(seed):
    urls = []
    try:

        splt_url = urlsplit(seed)
        newurl = splt_url.scheme +'://'+ splt_url.netloc 

        robots = Robots.fetch(newurl + '/robots.txt')
        agent = robots.agent('my-user-agent')
        
        if(agent.allowed(seed) != True):
            return -1

        
        
        fp = urllib.request.urlopen(seed)
        htmlcontent = fp.read()    
        clean_text = text_from_html(htmlcontent)
        

        invalid_ext = [".epub",".mobi", ".docx", ".doc", ".opf",
                       ".7z", ".ibooks", ".cbr", ".avi", ".mkv",
                       ".mp4", ".jpg", ".jpeg", ".png", ".gif" ,
                       ".pdf", ".iso", ".rar", ".tar", ".tgz", 
                       ".zip", ".dmg", ".exe", "torrent"]        
        
        soup = BeautifulSoup(htmlcontent, 'html.parser')
        
        for value in soup.find_all('a', href=True):
            
            ext_if_any = value['href'].split(".")[-1]
            
            if ext_if_any.lower() in  invalid_ext:
                continue
                
            valid = False
            for word in important_words:
                if str(value).lower().find(word.lower()) != -1:
                    valid = True
                    break
                    
            if valid == False:
                continue


            can_url = Canonicalization(seed, value['href'])

            if can_url == -1:
                continue

            if can_url.find("wikipedia") != -1 and can_url.split(".")[0][-2:] != 'en':
                continue

            if can_url.find("http://en.wikipedia.org/wiki/Book") != -1 :
                continue
                
            if can_url.find("http://en.wikipedia.org/wiki/Category") != -1 :
                continue

            if can_url.find("http://en.wikipedia.org/wiki/Portal") != -1:
                continue

            if can_url.find("https://donate.wikimedia.org") != -1:
                continue

            if can_url.find("http://en.wikipedia.org/wiki/Special:") != -1:
                continue

            if can_url.find("http://shop.wikimedia.org") != -1:
                continue

            if(can_url.find("https://pixnio.com")) != -1:
                continue
            
            if can_url.find("https://pixnio.com/uk/") != -1:
                continue
            
            if(can_url.find("http://www.public-domain-image.com/") != -1):
                continue
                
            urls.append(can_url)
    except:
        return -1
    
    return resp

In [9]:
clock_in_time = {
    
    "http://en.wikipedia.org/wiki/American_Revolutionary_War" : 0,
    "http://www.history.com/topics/american-revolution/american-revolution-history" : 0,
    "http://www.revolutionary-war.net/causes-of-the-american-revolution.html" : 0 
    
}

inlinks = {
    "http://en.wikipedia.org/wiki/American_Revolutionary_War" : [],
    "http://www.history.com/topics/american-revolution/american-revolution-history" : [],
    "http://www.revolutionary-war.net/causes-of-the-american-revolution.html" : []

}

outlinks = {
    "http://en.wikipedia.org/wiki/American_Revolutionary_War" : [],
    "http://www.history.com/topics/american-revolution/american-revolution-history" : [],
    "http://www.revolutionary-war.net/causes-of-the-american-revolution.html" : []

}

inlinkcount = {
    "http://en.wikipedia.org/wiki/American_Revolutionary_War" : 0,
    "http://www.history.com/topics/american-revolution/american-revolution-history" : 0,
    "http://www.revolutionary-war.net/causes-of-the-american-revolution.html" : 0
}

outlinkcounts = {
    "http://en.wikipedia.org/wiki/American_Revolutionary_War" : 0,
    "http://www.history.com/topics/american-revolution/american-revolution-history" : 0,
    "http://www.revolutionary-war.net/causes-of-the-american-revolution.html" : 0

}


frontier_System = {
     1: [ "http://en.wikipedia.org/wiki/American_Revolutionary_War",
          "http://www.history.com/topics/american-revolution/american-revolution-history",
          "http://www.revolutionary-war.net/causes-of-the-american-revolution.html"
        ]
}

current_level = 1
processed_doc = {}
added_doc = []
current_url = ""
old_url = ""
doc_count = 0
doc_id_map = {}
id_doc_map = {}
domain_parsed = []
banned_robot_urls = []

In [10]:
# splt_url = urlsplit("http://en.wikipedia.org/wiki/Main_Page")
# newurl = splt_url.scheme +'://'+ splt_url.netloc 
# if(newurl not in domain_parsed):
#     print(newurl)
# print(newurl)


# # This utility uses `requests` to fetch the content
# robots = Robots.fetch(newurl + '/robots.txt')

# # Get the rules for a specific agent
# agent = robots.agent('my-user-agent')
# agent.allowed("http://en.wikipedia.org/wiki/Main_Page")



In [ ]:
while(len(added_doc) < 20000 and frontier_System != {}):
    
    found_url = False
    current_level = 0
    for level in sorted(frontier_System.keys()):

        if(len(frontier_System[level]) == 0):
            frontier_System.pop(level, None)
            continue
        
        current_dict = {} 
        for url in frontier_System[level]:
            if url not in processed_doc:
                current_dict[url] = inlinkcount[url]
        
        current_level = level
        
        if(current_dict == {}):
            frontier_System.pop(level, None)
            continue
        


        dd = OrderedDict(sorted(current_dict.items(), key=lambda x: x[1]))
#         print(dd)
#         print(list(dd))
#         current_url = list(dd)[0] 
        
        max = dd[list(dd)[0]]
        current_url = list(dd)[0]
        for top_url in list(dd):
            if(dd[top_url]) == max and clock_in_time[top_url] < clock_in_time[current_url]:
                current_url = top_url
        
        processed_doc.append(current_url)
        frontier_System[level].remove(current_url)
        found_url = True
        
        break
    
    if(found_url == False):
        break

    resp = scrap(current_url)
    print(current_url)
    start_time = time.time()
    
    if(resp == -1):
        continue
        
    urls = resp['urls']


    added_doc.append(current_url)

    urls = list(set(urls))
    for url in urls:
        if url not in inlinkcount:
            inlinkcount[url] = 0
            inlinks[url] = []
            clock_in_time[url] = time.time()
        inlinkcount[url] = inlinkcount[url] + 1
        inlinks[url].append(current_url)
    outlinkcounts[current_url] = len(urls)
    outlinks[current_url] = urls
    
    if (current_level + 1) not in frontier_System.keys():
        frontier_System[current_level + 1] = []
   
    frontier_System[current_level + 1] = list(set(frontier_System[current_level + 1] + urls))
     
    doc_id_map[current_url] = doc_count
    id_doc_map[doc_count] = current_url
    doc_count = doc_count + 1
    
    
    resp['url'] = current_url
    
    file_name = "files/"+ str(doc_count) + ".json"
    with open(file_name, 'w') as outfile:
        json.dump(resp, outfile)
    
    
    now = time.time()
    diff  = (1- (time.time() - start_time))
    
    
    if(diff  > 0):
        time.sleep(diff)
        

http://www.time.com/time/specials/2007/top10/article/0,30583,1686204_1686252_1690928,00.html
http://en.wikipedia.org/wiki/Strawn-Wagner_Diamond
http://en.wikipedia.org/wiki/Di-positronium
http://en.wikipedia.org/wiki/White_dwarf
http://www.unileverusa.com/ourbrands/personalcare/vaseline.asp
http://www.nasa.gov/exploration/whyweexplore/Why_We_27_prt.htm
http://en.wikipedia.org/wiki/Heavy_water
http://en.wikipedia.org/wiki/Eta_meson
http://en.wikipedia.org/wiki/Arlo_U._Landolt
http://www.minorplanetcenter.net/iau/pressinfo/S1999J1.html
http://www.osti.gov/accomplishments/lederman.html
http://en.wikipedia.org/wiki/Carbon%E2%80%93fluorine_bond
https://www.nytimes.com/2006/03/02/science/02chamberlain.html
http://en.wikipedia.org/wiki/Upsilon_meson
http://adsabs.harvard.edu/full/seri/PASP./0026/0000197.000.html
http://en.wikipedia.org/wiki/File:Photograph_of_Cliff_Palace_at_Mesa_Verde_National_Park_-_NARA_-_544922.tif
http://www.rsc.org/chemistryworld/News/2007/September/13090701.asp
http://

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


http://education.jlab.org/beamsactivity/6thgrade/whatismatter/whatismatter.pdf
http://physicsworld.com/cws/article/news/25196
http://en.wikipedia.org/wiki/Leon_Lederman
http://en.wikipedia.org/wiki/Mark_R._Showalter
http://en.wikipedia.org/wiki/Joint_Institute_for_Nuclear_Research
http://en.wikipedia.org/wiki/ALVIN
http://en.wikipedia.org/wiki/Cretaceous_Period
http://news.bbc.co.uk/2/hi/science/nature/7340670.stm
http://en.wikipedia.org/wiki/George_Zweig
http://www.fnal.gov/pub/inquiring/physics/discoveries/top_quark.html
http://en.wikipedia.org/wiki/D0_experiment
http://web.mit.edu/newsoffice/2007/rna-history.html
http://en.wikipedia.org/wiki/Stark_Brothers_Nurseries_and_Orchards
http://doi.org/10.1126/science.204.4396.951
http://en.wikipedia.org/wiki/Heavy_hydrogen
http://en.wikipedia.org/wiki/Xi_baryon
http://en.wikipedia.org/wiki/Deuterium
http://www.astro.auth.gr/ANTIKATOPTRISMOI/nineplanets/nineplanets/larissa.html
http://en.wikipedia.org/wiki/Justus_von_Liebig
http://en.wikiped

http://nssdc.gsfc.nasa.gov/imgcat/html/object_page/vg2_1138920.html
http://www.britannica.com/EBchecked/topic/28507/antiproton
http://en.wikipedia.org/wiki/Russell_Hulse
http://www.solstation.com/stars/orcus.htm
http://en.wikipedia.org/wiki/Cosmic_microwave_background_radiation
http://en.wikipedia.org/wiki/Otto_Schaden
http://www.ieee.org/organizations/pubs/newsletters/emcs/fall06/franklin.pdf
http://www.azcentral.com/arizonarepublic/news/articles/0422lucy0422.html
http://www.cbat.eps.harvard.edu/iauc/03500/03532.html
http://en.wikipedia.org/wiki/Polyvinylidene_chloride
http://en.wikipedia.org/wiki/Barnard%27s_Star
http://en.wikipedia.org/wiki/John_Bell_Hatcher
http://www.stanford.edu/home/welcome/research/rem.html
http://en.wikipedia.org/wiki/Zener_diode
http://hyperphysics.phy-astr.gsu.edu/hbase/particles/xi.html
http://en.wikipedia.org/wiki/Harry_Coover
http://en.wikipedia.org/wiki/Chemical_potential
http://en.wikipedia.org/wiki/Charm_quark
http://en.wikipedia.org/wiki/Kaon
http://e

http://en.wikipedia.org/wiki/Presbyterian_Church_in_the_United_States_of_America
http://en.wikipedia.org/wiki/John_Witherspoon_College
http://www.johnwitherspooncollege.org/page/liberalarts.aspx
https://www.princeton.edu/pr/pwb/01/1105/1a.shtml
http://www.colonialhall.com/witherspoon/witherspoon.php
http://en.wikipedia.org/wiki/New_Jersey_Legislative_Council
http://en.wikipedia.org/wiki/Gifford,_East_Lothian
https://en.wikiquote.org/wiki/John_Witherspoon
https://web.archive.org/web/20150607151155/http:/www.ushistory.org/Declaration/signers/witherspoon.htm
http://en.wikipedia.org/wiki/Cosmology
https://web.archive.org/web/20150409102828/http:/www.johnwitherspooncollege.org/page/liberalarts.aspx
http://en.wikipedia.org/wiki/Doune_Castle
http://en.wikipedia.org/wiki/President%27s_House_(Princeton)
http://presbyterianhistory.com/b_witherspoon.htm
http://en.wikipedia.org/wiki/James_Beattie_(writer)
http://www.kittytours.org/thatman2/search.asp
http://en.wikipedia.org/wiki/Edmund_Lyndeck
htt

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


http://www.historicalartmedals.com/Medals%20of%20the%20Glorious%20Revolution%20.pdf
http://en.wikipedia.org/wiki/Omen
http://en.wikipedia.org/wiki/Charles_Talbot,_Duke_of_Shrewsbury
http://en.wikipedia.org/wiki/Socialist_Review
http://en.wikipedia.org/wiki/Declaration_of_Right
http://en.wikipedia.org/wiki/Heneage_Finch,_1st_Earl_of_Aylesford
http://en.wikipedia.org/wiki/Talk:Glorious_Revolution
http://en.wikipedia.org/wiki/Acclamation
http://en.wikipedia.org/wiki/Hounslow
http://en.wikipedia.org/wiki/Gaspar_Fagel
http://en.wikipedia.org/wiki/John_Hampden_(1653%E2%80%931696)
http://en.wikipedia.org/wiki/Supposititious_children
http://www.marxists.org/archive/hallas/works/1988/10/orange.htm
https://www.wikidata.org/wiki/Q189508
http://www.royal.gov.uk/HistoryoftheMonarchy/KingsandQueensoftheUnitedKingdom/TheStuarts/MaryIIWilliamIIIandTheActofSettlement/MaryIIWilliamIII.aspx
https://www.questia.com/read/5022714365
http://en.wikipedia.org/wiki/Everhard_van_Weede_Dijkvelt
http://www.worldca

http://en.wikipedia.org/wiki/Isolationist
http://www.dnaindia.com/india/report-a-women-s-movement-that-is-trying-to-bring-peace-to-israel-2434307
http://findarticles.com/p/articles/mi_gx5215/is_2002/ai_n19132457/pg_2
http://en.wikipedia.org/wiki/Peace_flag
http://en.wikipedia.org/wiki/UK_Trident_programme
http://en.wikipedia.org/wiki/Primitivism
http://en.wikipedia.org/wiki/Single-issue
http://en.wikipedia.org/wiki/Lawrence_S._Wittner
http://nzetc.victoria.ac.nz/tm/scholarly/tei-WH2-1Hom-c5.html
http://en.wikipedia.org/wiki/Open_government
https://www.wikidata.org/wiki/Q708312
http://en.wikipedia.org/wiki/National_Peace_Council
https://books.google.com/books/about/Peace.html
http://en.wikipedia.org/wiki/Concepcion_Picciotto
http://osulibrary.oregonstate.edu/specialcollections/coll/pauling/peace/narrative/page26.html
http://en.wikipedia.org/wiki/File:Settlements_map_eng.pdf
http://en.wikipedia.org/wiki/Proteomics
https://d-nb.info/gnd/4018520-5
http://en.wikipedia.org/wiki/Japanese_atta

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


http://bsmurthy.com/download/Mahatma_Gandhi_Leostoy_Letters_by_BSM.pdf
http://osulibrary.oregonstate.edu/specialcollections/coll/pauling/peace/narrative/page27.html
http://archives.theinternational.org.uk/2015/09/26/france-proves-incapable-of-learning-from-their-mistakes/
http://www.ppu.org.uk/learn/infodocs/people/pp-einstein2.html
http://en.wikipedia.org/wiki/Righteous_Among_the_Nations
http://en.wikipedia.org/wiki/Andr%C3%A9_and_Magda_Trocm%C3%A9
http://en.wikipedia.org/wiki/Peace_march
http://en.wikipedia.org/wiki/Janis_Karpinski
http://en.wikipedia.org/wiki/International_League_of_Humanists
http://www.thenation.com/article/spirit-june-12
http://en.wikipedia.org/wiki/Talk:Siege_of_Negapatam
https://www.wikidata.org/wiki/Special:EntityPage/Q7510266
http://en.wikipedia.org/wiki/Sadras
http://en.wikipedia.org/wiki/Talk:Battle_of_Connecticut_Farms
http://www.ctfarmscemetery.org/history.html
http://www.doublegv.com/ggv/battles/Springfld.html
https://www.wikidata.org/wiki/Special:EntityP

In [ ]:
# added_doc

In [ ]:
# while(len(added_doc) < 2):
#     old_url = current_url
#     level_at = -1
#     url_found = False
#     for level in sorted(frontier_System.keys()):

#         if(frontier_System[level] == {}):
#             frontier_System.pop(level, None)
#             continue
            
#         for url in frontier_System[level].keys():
#             if(url.find(current_domain) == -1):
#                 current_url = url
#                 level_at = level
#                 url_found = True
#                 current_domain = current_url.split("://",1)[1].split("/",1)[0]
#                 break
                
#         if(url_found == True):
#             break
            
# #     print(level_at)
# #     print(current_url,old_url)
# #     print(current_domain)
    
#     if(current_url == old_url):
        
#         if frontier_System == {}:
#             break
 
#         time.sleep(1)
#         level_at = (sorted(frontier_System.keys())[0])
#         current_url = list(frontier_System[level_at].keys())[0]
        
#     print(current_url)
#     frontier_System[level_at].pop(current_url, None)
        
# #     fp = urllib.request.urlopen("http://en.wikipedia.org/wiki/American_Revolutionary_War")
# #     htmlcontent = fp.read()


In [ ]:
# list(frontier_System[level_at].keys())[0]

In [ ]:
# frontier_System

In [ ]:
# current_url

In [ ]:
# frontier_System[level_at]

In [ ]:
# frontier_System

In [ ]:
# _

In [ ]:
# FrontierManagement = { 0 : }

In [ ]:
# # from urllib.request import urlopen
# # content = urlopen("https://www.quora.com/")


# import urllib.request
# req = urllib.request.Request("http://www.google.com", method="HEAD")

In [ ]:
# import urllib


# urobject = urllib.request.Request('https://en.wikipedia.org/wiki/Okapi_BM25', method="HEAD")

# resp = urllib.request.urlopen(urobject)
# a = resp.read()

# # print(a)
# print(resp.url)
# # # from bs4 import BeautifulSoup
# # # soup = BeautifulSoup(contents, 'html.parser')
# # # soup.find_all('a')

In [ ]:
# resp.url

In [ ]:
# import requests
# resp = requests.head('https://en.wikipedia.org/wiki/Okapi_BM25')

In [ ]:
# import requests
# requests.head('https://en.wikipedia.org/wiki/Okapi_BM25')
# _.headers

In [ ]:
# import time

In [ ]:
# start_time = time.time()
# for i in range(0,10):
#     now = time.time()
#     print(1- (now - start_time))
#     start_time = now